## Cost of Capital
##### A notebook by __Matt Harrington (@conda_env)__ for identifying optimal yield farm allocations.

In [1]:
## Fetch the data from the API and prints the last vault data

import requests
import json

# API endpoint
API_ENDPOINT = "https://vaults-dashboard-backend.fly.dev/getAllVaults?batch=1&input=%7B%220%22%3A%7B%22json%22%3A%7B%22interval%22%3A%227day%22%2C%22chainName%22%3Anull%7D%2C%22meta%22%3A%7B%22values%22%3A%7B%22chainName%22%3A%5B%22undefined%22%5D%7D%7D%7D%7D"

def get_vault_data():
    response = requests.get(API_ENDPOINT)
    if response.status_code == 200:
        return response.json()[0]['result']['data']['json']
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}")

# Fetch data from the API
vault_data = get_vault_data()
# We can print out the vault_data or inspect it to understand the structure and proceed
print(json.dumps(vault_data[-1], indent=2))

{
  "address": "0xf0b19f02c63d51b69563a2b675e0160e1c34397c",
  "asset": {
    "assetAddress": "0x4200000000000000000000000000000000000006",
    "decimals": 18,
    "name": "Wrapped Ether",
    "symbol": "WETH"
  },
  "name": "Pooltogether Prize WETH - Aave",
  "network": "optimism",
  "apy": 0,
  "rewards": [],
  "rewardsApy": null,
  "rawApy": 0,
  "tvlInUsd": 631781,
  "assetPriceInUsd": "237020000000",
  "maxDeposit": "0",
  "lendLink": "https://app.cabana.fi/vault/10/0xf0b19f02c63d51b69563a2b675e0160e1c34397c",
  "transactionCost": {
    "approve": 30000,
    "deposit": 320000,
    "withdraw": 340000,
    "claim": 210000
  },
  "tags": null,
  "protocolName": "pooltogether",
  "protocolVersion": "",
  "product": "pooltogether",
  "isCompounding": false,
  "warnings": []
}


### Optimize the allocation of capital across the vaults

Use cvxpy.py to perform the convex optimization along the yield curves.

In [2]:
import cvxpy as cp
import numpy as np
from pprint import pprint

# Constants
TAO = 1000000  # Total capital to be allocated
SLIPPAGE = 0.0001  # Slippage rate
MAX_VAULTS = 5  # Maximum number of vaults to allocate to
MAX_TVL_SHARE = 0.1  # Maximum share of TVL for allocation
HOLDING_DAYS = 30 # Number of days to hold the capital

# Fetch data from the API
vault_data = get_vault_data()

# Number of vaults
num_vaults = len(vault_data)

# Total Value Locked for each vault
tvls = np.array([vault['tvlInUsd'] for vault in vault_data])

# Optimization variables
allocations = cp.Variable(num_vaults)
selection = cp.Variable(num_vaults, boolean=True)

# Yield function for each vault (placeholder)
yields = cp.multiply(allocations, np.array([vault['apy'] for vault in vault_data]))

# Slippage adjusted yields
adjusted_yields = yields * (1 - SLIPPAGE)

# Objective: Maximize the total adjusted yield
objective = cp.Maximize(cp.sum(adjusted_yields))

# Constraints
constraints = [
    cp.sum(allocations) <= TAO,  # Total allocation must be TAO
    allocations <= selection * (0.5 * TAO),  # No more than half of TAO to any individual pool
    allocations >= 0,  # Allocations must be non-negative
    cp.sum(selection) <= MAX_VAULTS,  # No more than MAX_VAULTS vaults
    allocations <= tvls * MAX_TVL_SHARE,  # Allocation must be less than 10% of the TVL
]

# Problem
problem = cp.Problem(objective, constraints)

# Solve the problem using a solver that supports integer/binary programming
problem.solve(solver=cp.CBC)

# Check the status of the solution
if problem.status not in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
    print("Problem status:", problem.status)
else:
    # Optimal allocations
    optimal_allocations = allocations.value

allocated_vaults = np.where(optimal_allocations > 0)[0]

# Display the results
print("Optimal allocations:")
pprint({f"{vault_data[i]['name']}": 
    f"${optimal_allocations[i]/1_000:.2f}k" for i in allocated_vaults})

Optimal allocations:
{'Aave v2 USDC': '$263.22k',
 'Silo crvUSD (CRV collateral)': '$500.00k',
 'Yearn v3 DAI-A': '$54.82k',
 'Yearn v3 USDC.e-A': '$98.41k',
 'Yearn v3 USDT-A': '$83.56k'}


### Compose data & metadata into a single data table

In [3]:
import pandas as pd

n_allocations = len(allocated_vaults)

# Convert allocations to a numpy array for easier handling
optimal_allocations_array = np.array(optimal_allocations)

# Calculate the anticipated APY for each allocated vault
post_deposit_apys = []
for i in allocated_vaults:
    tvl_v = vault_data[i]["tvlInUsd"]
    apy_v = vault_data[i]["apy"] / 100
    allocation_v = optimal_allocations_array[i]
    adjusted_apy_v = apy_v * (tvl_v / (tvl_v + allocation_v))
    post_deposit_apys.append(adjusted_apy_v)

# Calculate the net APY for the portfolio
net_apy = np.average([
        post_deposit_apys[i] / 100 for i in range(n_allocations)
    ], weights=optimal_allocations_array[allocated_vaults])

# Calculate the expected USD return for each vault allocation
expected_yearly_usd_return = net_apy * sum(optimal_allocations_array[allocated_vaults])

# Prepare the data for the table
table_data = {
    "Protocol Name": [vault_data[i]["protocolName"] for i in allocated_vaults],
    "Network": [vault_data[i]["network"] for i in allocated_vaults],
    "Current APY": [f'{vault_data[i]["apy"] / 100}%' for i in allocated_vaults],
    "Post-Entry APY*": [f'{round(post_deposit_apys[i], 1)}%' for i in range(n_allocations)],
    "Asset Symbol": [vault_data[i]["asset"]["symbol"] for i in allocated_vaults],
    "Vault Allocation": [f'${round(optimal_allocations_array[i] / 1_000, 2)}k' for i in allocated_vaults],
    "Current TVL (USD)": [f'${round(vault_data[i]["tvlInUsd"] / 1_000_000, 2)}mm' for i in allocated_vaults],
    "URL Link": [vault_data[i]["lendLink"] for i in allocated_vaults],
    "Vault Address": [vault_data[i]["address"] for i in allocated_vaults],
}

# Create a DataFrame to display the table
vaults_table = pd.DataFrame(table_data)

# Print the expected USD return and net APY for the portfolio
print(f"Expected {HOLDING_DAYS}d USD return on the ${TAO / 1_000_000}mm " + 
    f"portfolio: ${round(expected_yearly_usd_return * (HOLDING_DAYS / 365) / 1_000, 2)}k")
print(f"Net APY for the portfolio: {round(net_apy * 100, 2)}%")

# Display the table
vaults_table

Expected 30d USD return on the $1.0mm portfolio: $17.27k
Net APY for the portfolio: 21.01%


,Protocol Name,Network,Current APY,Post-Entry APY*,Asset Symbol,Vault Allocation,Current TVL (USD),URL Link,Vault Address
0,aave,polygon,15.11%,14.9%,USDC,$263.22k,$24.01mm,https://app.aave.com/reserve-overview/?underly...,0x1a13F4Ca1d028320A707D99520AbFefca3998b7F
1,silo,mainnet,24.43%,24.3%,crvUSD,$500.0k,$67.93mm,https://app.silo.finance/silo/0x96eFdF95Cc47fe...,0xb27D1729489d04473631f0AFAca3c3A7389ac9F8
2,yearn,polygon,20.24%,18.4%,USDC,$98.41k,$0.98mm,https://yearn.fi/v3/137/0xA013Fbd4b711f9ded6fB...,0xA013Fbd4b711f9ded6fB09C1c0d358E2FbC2EAA0
3,yearn,polygon,31.35%,28.5%,DAI,$54.82k,$0.55mm,https://yearn.fi/v3/137/0x90b2f54C6aDDAD41b8f6...,0x90b2f54C6aDDAD41b8f6c4fCCd555197BC0F773B
4,yearn,polygon,20.77%,18.9%,USDT,$83.56k,$0.84mm,https://yearn.fi/v3/137/0xBb287E6017d3DEb0e2E6...,0xBb287E6017d3DEb0e2E65061e8684eab21060123
